### 숫자로 표현된 범주형 데이터

In [1]:
import pandas as pd
demo_df = pd.DataFrame({'숫자 특성': [0,1,2,1], 
                        '범주형 특성' : ['양말', '여우', '양말', '상자']})


In [2]:
print(pd.get_dummies(demo_df))

   숫자 특성  범주형 특성_상자  범주형 특성_양말  범주형 특성_여우
0      0          0          1          0
1      1          0          0          1
2      2          0          1          0
3      1          1          0          0


In [3]:
# 숫자 특성의 범주형 데이터를 문자타입으로 바꾼다
demo_df['숫자 특성'] = demo_df['숫자 특성'].astype(str)
pd.get_dummies(demo_df)

,숫자 특성_0,숫자 특성_1,숫자 특성_2,범주형 특성_상자,범주형 특성_양말,범주형 특성_여우
0,1,0,0,0,1,0
1,0,1,0,0,0,1
2,0,0,1,0,1,0
3,0,1,0,1,0,0


### OneHotEncoder

In [4]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse=False) 
# sparse=False로 설정하면 OneHotEncoder가 희소 행렬이 아니라 넘파이 배열을 반환
print(ohe.fit_transform(demo_df))

[[1. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 1.]
 [0. 0. 1. 0. 1. 0.]
 [0. 1. 0. 1. 0. 0.]]


In [9]:
print(ohe.get_feature_names_out())

['숫자 특성_0' '숫자 특성_1' '숫자 특성_2' '범주형 특성_상자' '범주형 특성_양말' '범주형 특성_여우']


### ColumnTransformer : 컬럼 별로 다르게 변환하기 (Preprocessing)

In [10]:
# adult dataset load하기
import os
import mglearn

data = pd.read_csv(
    os.path.join(mglearn.datasets.DATA_PATH, 'adult.data'),
    header=None, index_col=False,
    names=['age', 'workclass', 'fnlwgt', 'education', 'education-num','marital-status', 'occupation', 'relationship', 'race','gender', 'sex', 'capital-gain', 'capital-loss',
           'hour-per-week', 'native-country', 'income']
)

data = data[['age', 'workclass','education','gender', 'hour-per-week', 'occupation', 'income']]

In [11]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

ct = ColumnTransformer(
    [("scaling", StandardScaler(), ['age', 'hour-per-week']),
     ("onehot", OneHotEncoder(sparse=False),
      ['workclass', 'education','gender', 'occupation'])]
)


In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
# income을 제외한 모든 열을 추출한다
data_features = data.drop("income", axis=1)
# 데이터 프레임과 income을 분할
X_train, X_test, y_train, y_test = train_test_split(data_features, data.income, random_state=0)

ct.fit(X_train)
X_train_trans = ct.transform(X_train)
print(X_train_trans.shape)


ValueError: could not convert string to float: ' United-States'